# Install Whisper

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt install -y ffmpeg
!pip install -U openai-whisper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s

In [16]:
# copy the data to the local directory
!cp -r /content/drive/My\ Drive/files\ for\ slice\ long\ samples/* /content/

# parameters

In [17]:
WITH_TEAMIM = False
WITH_STRESS = False
SR = 16000
SLICE_TO_WAV = False

if WITH_TEAMIM and WITH_STRESS:
    print("WITH_TEAMIM and WITH_STRESS cannot be True at the same time.")
    exit(1)


# Get WhisperTimeSync

In [18]:
!rm -rf WhisperTimeSync
!git clone https://github.com/EtienneAb3d/WhisperTimeSync.git
import json

Cloning into 'WhisperTimeSync'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 219 (delta 44), reused 46 (delta 38), pack-reused 163
Receiving objects: 100% (219/219), 2.74 MiB | 25.74 MiB/s, done.
Resolving deltas: 100% (102/102), done.


# Transcribe

In [19]:
import librosa
import json
import io
from six.moves.urllib.request import urlopen
import pathlib
import soundfile as sf
from tqdm import tqdm
from nikud_and_teamim import remove_nikud, replace_teamim_with_emphasis, remove_nikud_and_teamim
import re

!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [20]:
!mkdir /content/texts_without_cantillations
!unzip -q /content/texts_without_cantillations.zip -d /content/texts_without_cantillations

In [21]:
import librosa
import requests
import json
import scipy.io.wavfile as wavfile
import io
from six.moves.urllib.request import urlopen
import pathlib
import soundfile as sf
from tqdm import tqdm
import re



In [ ]:
SR = 16000


books_data = [
    {"name": "Bereshit", "number": 1, "chapters": 50, "sefaria_name": "Genesis"},
    {"name": "Shemot", "number": 2, "chapters": 40, "sefaria_name": "Exodus"},
    {"name": "Vaikra", "number": 3, "chapters": 27, "sefaria_name": "Leviticus"},
    {"name": "Bamidbar", "number": 4, "chapters": 36, "sefaria_name": "Numbers"},
    {"name": "Dvarim", "number": 5, "chapters": 34, "sefaria_name": "Deuteronomy"}
]
sefer_names = ["", "בְּרֵאשִֽׁית", "שְׁמֽות", "וַיִּקְרָֽא", "בַּמִּדְבָּֽר", "דְּבָרִֽים"]
otiyot_gematria = ["", "אָֽלֶף", "בֵּֽית", "גִּֽימֶל", "דָּֽלֶת", "הֵֽא", "וָֽו", "זַֽיִן", "חֵֽית", "טֵֽית", "יֽוֹד", "יֽוֹד-אָֽלֶף", "יֽוֹד-בֵּֽית", "יֽוֹד-גִּֽימֶל", "יֽוֹד-דָּֽלֶת", "טֵֽית-וָֽו", "טֵֽית-זַֽיִן", "יֽ-זַֽיִן", "יֽוֹד-חֵֽית", "יֽוֹד-טֵֽית", "כַּֽף", "כַּֽף-אָֽלֶף", "כַּֽף-בֵּֽית", "כַּֽף-גִּֽימֶל", "כַּֽף-דָּֽלֶת", "כַּֽף-הֵֽא", "כַּֽף-וָֽו", "כַּֽף-זַֽיִן", "כַּֽף-חֵֽית", "כַּֽף-טֵֽית", "לָֽמֶד", "לָֽמֶד-אָֽלֶף", "לָֽמֶד-בֵּֽית", "לָֽמֶד-גִּֽימֶל", "לָֽמֶד-דָּֽלֶת", "לָֽמֶד-הֵֽא", "לָֽמֶד-וָֽו", "לָֽמֶד-זַֽיִן", "לָֽמֶד-חֵֽית", "לָֽמֶד-טֵֽית", "מֵֽם", "מֵֽם-אָֽלֶף", "מֵֽם-בֵּֽית", "מֵֽם-גִּֽימֶל", "מֵֽם-דָּֽלֶת", "מֵֽם-הֵֽא", "מֵֽם-וָֽו", "מֵֽם-זַֽיִן", "מֵֽם-חֵֽית", "מֵֽם-טֵֽית", "נֽוּן", "נֽוּן-אָֽלֶף", "נֽוּן-בֵּֽית", "נֽוּן-גִּֽימֶל", "נֽוּן-דָּֽלֶת", "נֽוּן-הֵֽא", "נֽוּן-וָֽו", "נֽוּן-זַֽיִן", "נֽוּן-חֵֽית", "נֽוּן-טֵֽית", "סָֽמֶךְ"]
otiyot_gematria = [remove_nikud(ot) for ot in otiyot_gematria]
dataset = {"text": [], "audio_file": []}

with open('links_for_audio_1.json', 'r') as f:
    links_for_audio = json.load(f)

# Calculate total number of chapters
total_chapters = sum(book["chapters"] for book in books_data)

error_chapters = []

# Create a progress bar
pbar = tqdm(total=total_chapters)

# Loop over all the chapters of all the books
for book in books_data:
    book_name = book["name"]
    for chapter in range(1, book["chapters"]+1):
        # check if the chapter already exists
        if pathlib.Path(f"/content/drive/MyDrive/cantilationless_audio/{book_name}_{chapter}.mp3").exists():
            pbar.update()
            continue
        # # test on small data
        # if chapter % 27 != 0: # test on small data
        #     continue
        url = links_for_audio["books"][book["number"]-1]["chapters"][chapter-1]["link"]
        # if get error in urlopen(url).read() we will add the chapter to error_chapters
        try:
            file = urlopen(url).read()
        except:
            error_chapters.append((book_name, chapter))
            print(f"All the chapters that we get error in urlopen(url).read() are: {error_chapters}")
            pbar.update()
            continue
        z = io.BytesIO(file)
        pathlib.Path((f"{book_name}_{chapter}.mp3")).write_bytes(z.getbuffer())
        audio, sr = librosa.load(f"{book_name}_{chapter}.mp3", sr=SR)
        # Get the text of the chapter from file
        with open(f"texts_without_cantillations/{book['name']}.{chapter}.txt", "r") as f:
            text = f.read()
        # Remove text in () and []:
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub(r'\[[^)]*\]', '', text)
        if WITH_TEAMIM:
            text = remove_nikud(text)
        else:
            if WITH_STRESS:
                text = remove_nikud(text)
                text = replace_teamim_with_emphasis(text)
            else:
                text = remove_nikud_and_teamim(text)

        text = f"{sefer_names[book['number']]} " + f"פרק {otiyot_gematria[chapter]} " + text
        with open(f"{book_name}_{chapter}.txt", "w") as f:
            f.write(text)
        # Transcribe the audio to get the timestamps (as srt file)
        !python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v3
        with open(f"{book_name}_{chapter}.srt", "r") as f:
            transcribed = f.read()
        if "תודה רבה" in transcribed:
            print("we get the error of 'תודה רבה', we will try with v2 model")
            # Remove the srt file
            !rm "{book_name}_{chapter}.srt"
            !python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v2
            with open(f"{book_name}_{chapter}.srt", "r") as f:
                transcribed = f.read()
            if "תודה רבה" in transcribed:
                print("!!!!! we get the error of 'תודה רבה', also with v2 model !!!!!!!!")
                !rm "{book_name}_{chapter}.srt"
                print("we will try with v1 model")
                !python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large
                with open(f"{book_name}_{chapter}.srt", "r") as f:
                    transcribed = f.read()

        # Align(sync) the text with the audio
        !java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar /content/"{book_name}_{chapter}.srt" /content/"{book_name}_{chapter}.txt" he
        with open(f"{book_name}_{chapter}.txt.srt", "r") as f:
            srt = f.read()
        if SLICE_TO_WAV:
            for i, line in enumerate(srt.split("\n")):
                if i % 4 == 0: # The line is a number of the subtitle
                    continue
                if i % 4 == 1: # The line is a time
                    start, end = line.split(" --> ")
                    start = start.replace(",", ".")
                    end = end.replace(",", ".")
                    start = start.split(":")
                    end = end.split(":")
                    start = float(start[0])*3600 + float(start[1])*60 + float(start[2])
                    end = float(end[0])*3600 + float(end[1])*60 + float(end[2])
                    start = float(start)
                    end = float(end)
                    continue
                if i % 4 == 2: # The line is the text
                    dataset["text"].append(line)
                    dataset["audio_file"].append(f"{book_name}_{chapter}_{int(i/4):02}.wav")
                    # Save the audio file
                    sf.write(f"{book_name}_{chapter}_{int(i/4):02}.wav", audio[int(start*SR):int(end*SR)], SR)
                if i % 4 == 3: # The line is an empty line
                    continue
            # Remove the original audio file and the srt file
            !rm "{book_name}_{chapter}.txt.srt"
            !rm "{book_name}_{chapter}.mp3"

        # Remove the text file and the srt(low quality subtitles) file
        !rm "{book_name}_{chapter}.srt"
        !rm "{book_name}_{chapter}.txt"

        # Move the srt file and the audio file to the correct folder
        !mv "{book_name}_{chapter}.txt.srt" /content/drive/MyDrive/cantilationless_srt/"{book_name}_{chapter}".srt
        !mv "{book_name}_{chapter}.mp3" /content/drive/MyDrive/cantilationless_audio/"{book_name}_{chapter}".mp3


        # Update the progress bar
        pbar.update()

# Save the dataset
with open("dataset.json", "w") as f:
    json.dump(dataset, f)
print("Done")


 16%|█▌        | 30/187 [01:49<09:34,  3.66s/it]


Loading Whisper model: large-v3 ...
 31%|███████████▋                          | 906M/2.88G [00:35<00:47, 45.0MiB/s]

In [ ]:

book_name = "Bereshit"
chapter = 27
url = links_for_audio["books"][1]["chapters"][chapter-1]["link"]
z = io.BytesIO(urlopen(url).read())
pathlib.Path((f"{book_name}_{chapter}.mp3")).write_bytes(z.getbuffer())
!python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v3

In [ ]:
# Save the dataset
with open("dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f)

In [7]:
# play random audio and print its text
import random
import IPython.display as ipd
index = random.randint(0, len(dataset["audio_file"]))
print(dataset["text"][index])
ipd.Audio(dataset["audio_file"][index])


NameError: name 'dataset' is not defined

In [ ]:
index = 14
print(dataset["text"][index])
ipd.Audio(dataset["audio_file"][index])

In [8]:
# Sync the times of the real text using the srt file
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar "{book_name}_{chapter}.srt" "{book_name}_{chapter}.txt" he

java.io.FileNotFoundException: {book_name}_{chapter}.srt (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at com.cubAIx.WhisperTimeSync.WhisperTimeSync.load(WhisperTimeSync.java:20)
	at com.cubAIx.WhisperTimeSync.WhisperTimeSync.processFile(WhisperTimeSync.java:45)
	at com.cubAIx.WhisperTimeSync.WhisperTimeSync.main(WhisperTimeSync.java:120)


In [9]:
!cat /content/51.Shmot_1.srt

cat: /content/51.Shmot_1.srt: No such file or directory


# Synchronize

In [ ]:
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar /content/51.Shmot_1.srt /content/Exodus.1.txt he

In [ ]:
!cat /content/Exodus.1.txt.srt